In [2]:
!pip install torch torchvision torchaudio
!pip install torch-geometric



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 803.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
import torch
from torch_geometric.data import Data

In [4]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')


Processing...
Done!


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.948481559753418
Epoch 2, Loss: 1.862858772277832
Epoch 3, Loss: 1.7502641677856445
Epoch 4, Loss: 1.6401373147964478
Epoch 5, Loss: 1.5111979246139526
Epoch 6, Loss: 1.3893908262252808
Epoch 7, Loss: 1.2914828062057495
Epoch 8, Loss: 1.1358057260513306
Epoch 9, Loss: 1.0419921875
Epoch 10, Loss: 0.8974969983100891
Epoch 11, Loss: 0.8345098495483398
Epoch 12, Loss: 0.7416336536407471
Epoch 13, Loss: 0.6646862626075745
Epoch 14, Loss: 0.5565491914749146
Epoch 15, Loss: 0.5173380970954895
Epoch 16, Loss: 0.4519515931606293
Epoch 17, Loss: 0.38276341557502747
Epoch 18, Loss: 0.33981502056121826
Epoch 19, Loss: 0.2992097735404968
Epoch 20, Loss: 0.29955118894577026
Epoch 21, Loss: 0.31198781728744507
Epoch 22, Loss: 0.2114304155111313
Epoch 23, Loss: 0.19332531094551086
Epoch 24, Loss: 0.16351166367530823
Epoch 25, Loss: 0.16651928424835205
Epoch 26, Loss: 0.15282106399536133
Epoch 27, Loss: 0.13675978779792786
Epoch 28, Loss: 0.12937752902507782
Epoch 29, Loss: 0.119550704

In [7]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
accuracy = correct / int(data.test_mask.sum())
print(f'Accuracy: {accuracy}')


Accuracy: 0.8


In [8]:
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [9]:
model = GAT(dataset.num_node_features, dataset.num_classes).to(device)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 2.053623676300049
Epoch 2, Loss: 2.0357770919799805
Epoch 3, Loss: 1.995967149734497
Epoch 4, Loss: 2.018946409225464
Epoch 5, Loss: 2.021136999130249
Epoch 6, Loss: 2.004504442214966
Epoch 7, Loss: 1.9476691484451294
Epoch 8, Loss: 2.022987127304077
Epoch 9, Loss: 2.0058791637420654
Epoch 10, Loss: 2.0157382488250732
Epoch 11, Loss: 2.0013160705566406
Epoch 12, Loss: 2.078397750854492
Epoch 13, Loss: 1.9793949127197266
Epoch 14, Loss: 1.9922879934310913
Epoch 15, Loss: 2.034505844116211
Epoch 16, Loss: 1.975099802017212
Epoch 17, Loss: 2.019556760787964
Epoch 18, Loss: 1.9973750114440918
Epoch 19, Loss: 1.9941303730010986
Epoch 20, Loss: 2.0310449600219727
Epoch 21, Loss: 1.9679110050201416
Epoch 22, Loss: 1.984559416770935
Epoch 23, Loss: 1.968863606452942
Epoch 24, Loss: 2.0748863220214844
Epoch 25, Loss: 2.1042873859405518
Epoch 26, Loss: 1.9710577726364136
Epoch 27, Loss: 1.9721705913543701
Epoch 28, Loss: 1.9819270372390747
Epoch 29, Loss: 2.015488624572754
Epoch 3

In [10]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
accuracy = correct / int(data.test_mask.sum())
print(f'Accuracy: {accuracy}')


Accuracy: 0.134
